In [7]:
## Model training

In [5]:
import pandas as pd

In [6]:
df=pd.read_csv('gemstone.csv')
df=df.head(12000)
df.head()

,id,carat,cut,color,clarity,depth,table,x,y,z,price
0,0,1.52,Premium,F,VS2,62.2,58.0,7.27,7.33,4.55,13619
1,1,2.03,Very Good,J,SI2,62.0,58.0,8.06,8.12,5.05,13387
2,2,0.70,Ideal,G,VS1,61.2,57.0,5.69,5.73,3.50,2772
3,3,0.32,Ideal,G,VS1,61.6,56.0,4.38,4.41,2.71,666
4,4,1.70,Premium,G,VS2,62.6,59.0,7.65,7.61,4.77,14453


In [7]:
df=df.drop(labels=['id'],axis=1)

In [8]:
## Independent and dependent features
X = df.drop(labels=['price'],axis=1)
Y = df[['price']]

In [9]:
#Define categorical and numerical columns (for encoding and scaling)
categorical_cols=X.select_dtypes(include='object').columns
numerical_cols=X.select_dtypes(exclude='object').columns

In [10]:
cut_categories=['Fair','Good','Very Good','Premium','Ideal']
color_categories=['D','E','F','G','H','I','J']
clarity_categories=['I1','SI2','SI1','VS2','VS1','VVS2','VVS1','IF']

In [11]:
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import OrdinalEncoder

from sklearn.pipeline import Pipeline
from sklearn.compose import ColumnTransformer

In [12]:
#Numerical Pipeline
num_pipeline=Pipeline(
    steps=[
        ('imputer',SimpleImputer(strategy='median')),
        ('scaler',StandardScaler())
    ]

)

#Categorical Pipeline
cat_pipeline=Pipeline(
    steps=[
        ('imputer',SimpleImputer(strategy='most_frequent')),
        ('encoder',OrdinalEncoder(categories=[cut_categories,color_categories,clarity_categories])),
        ('scaler',StandardScaler())

    ]
)

preprocessor=ColumnTransformer([
('num_pipeline',num_pipeline,numerical_cols),
('cat_pipeline',cat_pipeline,categorical_cols)
    
])

In [13]:
## Train test split

from sklearn.model_selection import train_test_split

X_train,X_test,y_train,y_test=train_test_split(X,Y,test_size=30,random_state=42)

In [14]:
X_train=pd.DataFrame(preprocessor.fit_transform(X_train),columns=preprocessor.get_feature_names_out())
X_test=pd.DataFrame(preprocessor.transform(X_test),columns=preprocessor.get_feature_names_out())

In [15]:
X_train.head()

,num_pipeline__carat,num_pipeline__depth,num_pipeline__table,num_pipeline__x,num_pipeline__y,num_pipeline__z,cat_pipeline__cut,cat_pipeline__color,cat_pipeline__clarity
0,0.231735,1.016410,0.402782,0.352553,0.395144,0.472879,-1.152047,0.303526,0.012187
1,-0.797610,-1.032030,-0.635832,-0.775480,-0.757742,-0.859994,0.865805,-0.314156,1.332341
2,1.518418,0.550856,0.922088,1.408965,1.457961,1.487021,-0.143121,0.303526,-0.647890
3,-1.033502,-0.659586,0.402782,-1.223112,-1.199081,-1.251163,-0.143121,0.921208,-0.647890
4,1.561307,0.923299,-0.635832,1.426871,1.484982,1.559460,0.865805,0.921208,1.332341


In [16]:
from sklearn.linear_model import LinearRegression,Lasso,Ridge,ElasticNet
from xgboost import XGBRegressor
from sklearn.metrics import r2_score,mean_absolute_error,mean_squared_error

In [17]:
regression=LinearRegression()
regression.fit(X_train,y_train)

LinearRegression()

In [18]:
regression.coef_

array([[ 6539.44727703,   -67.32590191,   -42.42632335, -2182.1642553 ,
          386.50257949,  -551.19715511,    84.53318693,  -453.54758523,
          662.17108684]])

In [19]:
regression.intercept_

array([3995.19874687])

In [20]:
import numpy as np
def evaluate_model(true, predicted):
    mae=mean_absolute_error(true, predicted)
    mse=mean_squared_error(true, predicted)
    rmse=np.sqrt(mean_squared_error(true, predicted))
    r_square=r2_score(true, predicted)
    return mae,rmse,r_square

In [23]:
models={
    'LinearRegression':LinearRegression(),
    "Lasso":Lasso(),
    "Ridge":Ridge(),
    "ElasticNet":ElasticNet(),
    "XGBoost":XGBRegressor()
    }
trained_model_list=[]
model_list=[]
r2_list=[]

for i in range(len(list(models))):
    model=list(models.values())[i]
    model.fit(X_train,y_train)

    y_pred=model.predict(X_test)

    mae,rmse,r_square=evaluate_model(y_test,y_pred)

    print(list(models.keys())[i])
    model_list.append(list(models.keys())[i])

    print('Model training performance')
    print('RMSE:',rmse)
    print("MAE:",mae)
    print("R2 score: ",r_square*100,"%")

    r2_list.append(r_square)

    print('='*35)
    print('\n')

LinearRegression
Model training performance
RMSE: 758.4930547750009
MAE: 591.7218306655307
R2 score:  96.76009535410121 %


Lasso
Model training performance
RMSE: 758.2991478871403
MAE: 592.0090724929472
R2 score:  96.76175168969694 %


Ridge
Model training performance
RMSE: 757.9616805728679
MAE: 591.6318261326793
R2 score:  96.76463329597904 %


ElasticNet
Model training performance
RMSE: 1402.3078876211462
MAE: 1017.4908797470416
R2 score:  88.92571323247292 %


XGBoost
Model training performance
RMSE: 624.80102255209
MAE: 343.5966837565104
R2 score:  97.80157084491552 %




In [24]:
model_list

['LinearRegression', 'Lasso', 'Ridge', 'ElasticNet', 'XGBoost']